In [4]:
import retro,time

In [2]:
env = retro.make(game='StreetFighterIISpecialChampionEdition-Genesis')
env.observation_space.sample(),env.action_space.sample()

(array([[[255, 212, 203],
         [ 92, 255, 119],
         [149, 101, 170],
         ...,
         [100, 232, 148],
         [202,  39,  92],
         [ 22, 113, 128]],
 
        [[144, 119,  68],
         [ 60,  68, 224],
         [134,  42, 166],
         ...,
         [146,   5, 229],
         [141,  78,  34],
         [133, 251, 145]],
 
        [[153,  78,  20],
         [ 56, 224, 239],
         [ 80,  73, 251],
         ...,
         [115,  45,  91],
         [104,  41,  11],
         [  2, 167,  10]],
 
        ...,
 
        [[200,  90, 207],
         [ 13,  20,  78],
         [144,  51,  92],
         ...,
         [ 83,   5, 233],
         [210, 183, 135],
         [247, 224,  60]],
 
        [[ 64,   4,   9],
         [170, 162,  18],
         [101,  38, 109],
         ...,
         [154,  66, 214],
         [128, 253,  43],
         [140,  28,  26]],
 
        [[ 36, 106,  78],
         [  7, 118, 216],
         [ 33, 115,  35],
         ...,
         [ 84, 106,   5],
  

# Random environment

In [3]:
obs = env.reset()
# Set flag to flase
done = False
for game in range(1): 
    while not done: 
        if done: 
            obs = env.reset()
        env.render()
        obs, reward, done, info = env.step(env.action_space.sample())
#         time.sleep(0.01)
        print(reward)
print(info)

NameError: name 'env' is not defined

# Custom Environment

In [5]:
from gym import Env
from gym.spaces import MultiBinary, Box
import numpy as np
import cv2
from matplotlib import pyplot as plt

In [3]:
# 1. Frame
# 2. preprocess  200x256x3 > 84x84x1
# 3. change in pixels current_frame - the last frame

In [4]:
# gray = cv2.cvtColor(obs,cv2.COLOR_BGR2GRAY)
# resize = cv2.resize(gray,(84,84),cv2.INTER_CUBIC)
# channels = np.reshape(resize,(84,84,1))
# plt.imshow(cv2.cvtColor(channels, cv2.COLOR_BGR2RGB))

In [8]:
#custom environment

class StreetFighter(Env):
    def __init__(self):
        super().__init__()
        self.observation_space = Box(low=0, high=255, shape=(84, 84, 1), dtype=np.uint8)
        self.action_space = MultiBinary(12)
        self.game = retro.make(game='StreetFighterIISpecialChampionEdition-Genesis', use_restricted_actions=retro.Actions.FILTERED,players = 2)
        #self.score = 0
    
    def step(self, action):
        obs, reward, done, info = self.game.step(action)
        obs = self.preprocess(obs)
        
        # Preprocess frame from game
        frame_delta = obs 
#         - self.previous_frame
#         self.previous_frame = obs 
        
        # Shape reward
        # reward = info['score'] - self.score 
        # self.score = info['score']
        reward = (self.enemy_health - info['enemy_health'])*2 + (info['health'] - self.health)

        return frame_delta, reward, done, info 
    
    def render(self, *args, **kwargs): 
        self.game.render(*args, **kwargs)
    
    def reset(self):
        self.previous_frame = np.zeros(self.game.observation_space.shape)
        
        # Frame delta
        obs = self.game.reset()
        obs = self.preprocess(obs)
        self.previous_frame = obs
        self.health = 176
        self.enemy_health = 176
        
        # Create initial variables
        self.score = 0

        return obs
    
    def preprocess(self, observation): 
        gray = cv2.cvtColor(observation, cv2.COLOR_BGR2GRAY)
        resize = cv2.resize(gray, (84,84), interpolation=cv2.INTER_CUBIC)
        state = np.reshape(resize, (84,84,1))
        return state
    
    def close(self): 
        self.game.close()

In [6]:
# env.close()

In [ ]:
env = StreetFighter()

In [ ]:
# Reset game to starting state
obs = env.reset()
# Set flag to flase
done = False
for game in range(1): 
    while not done: 
        if done: 
            obs = env.reset()
        env.render()
        obs, reward, done, info = env.step(env.action_space.sample())
        time.sleep(0.01)
        if reward > 0: 
            print(reward)

In [ ]:
#Frame Delta
obs, reward, done, info = env.step(env.action_space.sample())
plt.imshow(cv2.cvtColor(obs, cv2.COLOR_BGR2RGB))

In [ ]:
!pip3 install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu116

In [ ]:
!pip install stable-baselines3[extra] optuna

In [9]:
# Importing the optimzation frame - HPO
import optuna
# PPO algo for RL
from stable_baselines3 import PPO
# Bring in the eval policy method for metric calculation
from stable_baselines3.common.evaluation import evaluate_policy
# Import the sb3 monitor for logging 
from stable_baselines3.common.monitor import Monitor
# Import the vec wrappers to vectorize and frame stack
from stable_baselines3.common.vec_env import DummyVecEnv, VecFrameStack
# Import os to deal with filepaths
import os

In [10]:
LOG_DIR = './logs/'
OPT_DIR = './opt/'

In [11]:
# Function to return test hyperparameters - define the object function
def optimize_ppo(trial): 
    return {
        'n_steps':trial.suggest_int('n_steps', 2048, 8192),
        'gamma':trial.suggest_loguniform('gamma', 0.8, 0.9999),
        'learning_rate':trial.suggest_loguniform('learning_rate', 1e-5, 1e-4),
        'clip_range':trial.suggest_uniform('clip_range', 0.1, 0.4),
        'gae_lambda':trial.suggest_uniform('gae_lambda', 0.8, 0.99)
    }

In [12]:
SAVE_PATH = os.path.join(OPT_DIR, 'trial_{}_best_model'.format(1))

In [13]:
# Run a training loop and return mean reward 
def optimize_agent(trial):
    try:
        model_params = optimize_ppo(trial) 

        # Create environment 
        env = StreetFighter()
        env = Monitor(env, LOG_DIR)
        env = DummyVecEnv([lambda: env])
        env = VecFrameStack(env, 4, channels_order='last')

        # Create algo 
        model = PPO('CnnPolicy', env, tensorboard_log=LOG_DIR, verbose=0, **model_params)
        model.learn(total_timesteps=20000)
        #model.learn(total_timesteps=100000)

        # Evaluate model 
        mean_reward, _ = evaluate_policy(model, env, n_eval_episodes=3)
        env.close()

        SAVE_PATH = os.path.join(OPT_DIR, 'trial_{}_best_model'.format(trial.number))
        model.save(SAVE_PATH)

        return mean_reward

    except Exception as e:
        return -1000

In [14]:
# Creating the experiment 
study = optuna.create_study(direction='maximize')
study.optimize(optimize_agent, n_trials=10, n_jobs=1)
#study.optimize(optimize_agent, n_trials=100, n_jobs=1)

[I 2022-12-14 22:26:19,556] A new study created in memory with name: no-name-c65e2746-8001-4521-8e97-2a445681c105
C:\Users\RUSHIT\AppData\Local\Temp\ipykernel_15236\3802630500.py:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'gamma':trial.suggest_loguniform('gamma', 0.8, 0.9999),
C:\Users\RUSHIT\AppData\Local\Temp\ipykernel_15236\3802630500.py:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'learning_rate':trial.suggest_loguniform('learning_rate', 1e-5, 1e-4),
C:\Users\RUSHIT\AppData\Local\Temp\ipykernel_15236\3802630500.py:7: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://g

KeyboardInterrupt: 

In [10]:
study.best_params, study

({'n_steps': 4092,
  'gamma': 0.8469613057339491,
  'learning_rate': 1.688874996274341e-05,
  'clip_range': 0.186254787282314,
  'gae_lambda': 0.8516711838664867},
 <optuna.study.study.Study at 0x1fc4f8f1ca0>)

In [11]:
study.best_trial

FrozenTrial(number=0, values=[70300.0], datetime_start=datetime.datetime(2022, 12, 4, 19, 55, 38, 215985), datetime_complete=datetime.datetime(2022, 12, 4, 20, 12, 22, 664443), params={'n_steps': 4092, 'gamma': 0.8469613057339491, 'learning_rate': 1.688874996274341e-05, 'clip_range': 0.186254787282314, 'gae_lambda': 0.8516711838664867}, distributions={'n_steps': IntDistribution(high=8192, log=False, low=2048, step=1), 'gamma': FloatDistribution(high=0.9999, log=True, low=0.8, step=None), 'learning_rate': FloatDistribution(high=0.0001, log=True, low=1e-05, step=None), 'clip_range': FloatDistribution(high=0.4, log=False, low=0.1, step=None), 'gae_lambda': FloatDistribution(high=0.99, log=False, low=0.8, step=None)}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=0, state=TrialState.COMPLETE, value=None)

In [11]:
model = PPO.load(os.path.join(OPT_DIR, 'trial_9_best_model.zip'))

# Setup Callback

In [15]:
# Import base callback 
from stable_baselines3.common.callbacks import BaseCallback

In [14]:
!jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok


In [15]:
!pip install ipywidgets

In [16]:
class TrainAndLoggingCallback(BaseCallback):

    def __init__(self, check_freq, save_path, verbose=1):
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path

    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path, 'best_model_{}'.format(self.n_calls))
            self.model.save(model_path)

        return True

In [13]:
CHECKPOINT_DIR = './train/'

In [14]:
callback = TrainAndLoggingCallback(check_freq=1000, save_path=CHECKPOINT_DIR)

# Train model

In [15]:
# Create environment 
env = StreetFighter()
env = Monitor(env, LOG_DIR)
env = DummyVecEnv([lambda: env])
env = VecFrameStack(env, 4, channels_order='last')

In [16]:
model_params = {'n_steps': 4092,
  'gamma': 0.8469613057339491,
  'learning_rate': 1.688874996274341e-05,
  'clip_range': 0.186254787282314,
  'gae_lambda': 0.8516711838664867}
model_params['n_steps'] = 4032  # set n_steps to 7488 or a factor of 64
# model_params['learning_rate'] = 5e-7
model_params

{'n_steps': 4032,
 'gamma': 0.8469613057339491,
 'learning_rate': 1.688874996274341e-05,
 'clip_range': 0.186254787282314,
 'gae_lambda': 0.8516711838664867}

In [17]:
model = PPO('CnnPolicy', env, tensorboard_log=LOG_DIR, verbose=1, **model_params)

Using cpu device
Wrapping the env in a VecTransposeImage.


In [18]:
# Reload previous weights from HPO
model.load(os.path.join(OPT_DIR, 'trial_0_best_model.zip'))

In [ ]:
# Kick off training 
model.learn(total_timesteps=5000000, callback=callback)
# model.learn(total_timestep=5000000) 

Logging to ./logs/PPO_47
-----------------------------
| time/              |      |
|    fps             | 106  |
|    iterations      | 1    |
|    time_elapsed    | 37   |
|    total_timesteps | 4032 |
-----------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 5.9e+03      |
|    ep_rew_mean          | 6e+03        |
| time/                   |              |
|    fps                  | 43           |
|    iterations           | 2            |
|    time_elapsed         | 186          |
|    total_timesteps      | 8064         |
| train/                  |              |
|    approx_kl            | 0.0045748125 |
|    clip_fraction        | 0.0291       |
|    clip_range           | 0.186        |
|    entropy_loss         | -8.32        |
|    explained_variance   | -0.000798    |
|    learning_rate        | 1.69e-05     |
|    loss                 | 70.4         |
|    n_updates            | 10    

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 8.55e+03    |
|    ep_rew_mean          | 2.98e+04    |
| time/                   |             |
|    fps                  | 31          |
|    iterations           | 11          |
|    time_elapsed         | 1411        |
|    total_timesteps      | 44352       |
| train/                  |             |
|    approx_kl            | 0.010029152 |
|    clip_fraction        | 0.106       |
|    clip_range           | 0.186       |
|    entropy_loss         | -8.16       |
|    explained_variance   | 0.00603     |
|    learning_rate        | 1.69e-05    |
|    loss                 | 1.93e+03    |
|    n_updates            | 100         |
|    policy_gradient_loss | -0.00407    |
|    value_loss           | 6.13e+04    |
-----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 8.55e+

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 1.55e+04    |
|    ep_rew_mean          | 5.11e+04    |
| time/                   |             |
|    fps                  | 29          |
|    iterations           | 21          |
|    time_elapsed         | 2834        |
|    total_timesteps      | 84672       |
| train/                  |             |
|    approx_kl            | 0.012998974 |
|    clip_fraction        | 0.196       |
|    clip_range           | 0.186       |
|    entropy_loss         | -7.87       |
|    explained_variance   | 0.00141     |
|    learning_rate        | 1.69e-05    |
|    loss                 | 2.09        |
|    n_updates            | 200         |
|    policy_gradient_loss | 0.00302     |
|    value_loss           | 461         |
-----------------------------------------
----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 1.55e+04

----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 1.27e+04   |
|    ep_rew_mean          | 4.21e+04   |
| time/                   |            |
|    fps                  | 30         |
|    iterations           | 31         |
|    time_elapsed         | 4079       |
|    total_timesteps      | 124992     |
| train/                  |            |
|    approx_kl            | 0.18396144 |
|    clip_fraction        | 0.608      |
|    clip_range           | 0.186      |
|    entropy_loss         | -7.01      |
|    explained_variance   | 0.122      |
|    learning_rate        | 1.69e-05   |
|    loss                 | 494        |
|    n_updates            | 300        |
|    policy_gradient_loss | 0.0407     |
|    value_loss           | 2.01e+04   |
----------------------------------------
--------------------------------------
| rollout/                |          |
|    ep_len_mean          | 1.27e+04 |
|    ep_rew_mean      

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 1.17e+04    |
|    ep_rew_mean          | 3.42e+04    |
| time/                   |             |
|    fps                  | 31          |
|    iterations           | 41          |
|    time_elapsed         | 5217        |
|    total_timesteps      | 165312      |
| train/                  |             |
|    approx_kl            | 0.045118287 |
|    clip_fraction        | 0.414       |
|    clip_range           | 0.186       |
|    entropy_loss         | -6.21       |
|    explained_variance   | 0.298       |
|    learning_rate        | 1.69e-05    |
|    loss                 | 382         |
|    n_updates            | 400         |
|    policy_gradient_loss | 0.0218      |
|    value_loss           | 3.92e+03    |
-----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 1.17e+

...

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 5.76e+03    |
|    ep_rew_mean          | 1.75e+03    |
| time/                   |             |
|    fps                  | 37          |
|    iterations           | 371         |
|    time_elapsed         | 40276       |
|    total_timesteps      | 1495872     |
| train/                  |             |
|    approx_kl            | 0.007326748 |
|    clip_fraction        | 0.072       |
|    clip_range           | 0.186       |
|    entropy_loss         | -0.98       |
|    explained_variance   | -5.75       |
|    learning_rate        | 1.69e-05    |
|    loss                 | -0.0109     |
|    n_updates            | 3700        |
|    policy_gradient_loss | -0.0113     |
|    value_loss           | 0.0289      |
-----------------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 5.76

----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 5.82e+03   |
|    ep_rew_mean          | 1.77e+03   |
| time/                   |            |
|    fps                  | 37         |
|    iterations           | 381        |
|    time_elapsed         | 41322      |
|    total_timesteps      | 1536192    |
| train/                  |            |
|    approx_kl            | 0.00812065 |
|    clip_fraction        | 0.0783     |
|    clip_range           | 0.186      |
|    entropy_loss         | -0.638     |
|    explained_variance   | 0.00653    |
|    learning_rate        | 1.69e-05   |
|    loss                 | 70.7       |
|    n_updates            | 3800       |
|    policy_gradient_loss | -0.00364   |
|    value_loss           | 636        |
----------------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 5.82e+03     |
|    ep_re

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 5.85e+03    |
|    ep_rew_mean          | 1.80e+03    |
| time/                   |             |
|    fps                  | 37          |
|    iterations           | 391         |
|    time_elapsed         | 42364       |
|    total_timesteps      | 1576512     |
| train/                  |             |
|    approx_kl            | 0.009839978 |
|    clip_fraction        | 0.0677      |
|    clip_range           | 0.186       |
|    entropy_loss         | -0.508      |
|    explained_variance   | 0.136       |
|    learning_rate        | 1.69e-05    |
|    loss                 | 0.196       |
|    n_updates            | 3900        |
|    policy_gradient_loss | -0.00378    |
|    value_loss           | 457         |
-----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 5.86e+

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 5.86e+03    |
|    ep_rew_mean          | 1.81e+03    |
| time/                   |             |
|    fps                  | 37          |
|    iterations           | 401         |
|    time_elapsed         | 43593       |
|    total_timesteps      | 1616832     |
| train/                  |             |
|    approx_kl            | 0.017542314 |
|    clip_fraction        | 0.143       |
|    clip_range           | 0.186       |
|    entropy_loss         | -0.534      |
|    explained_variance   | 0.0225      |
|    learning_rate        | 1.69e-05    |
|    loss                 | 533         |
|    n_updates            | 4000        |
|    policy_gradient_loss | 0.00481     |
|    value_loss           | 1.11e+03    |
-----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 5.86e+

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 5.92e+03    |
|    ep_rew_mean          | 1.86e+03    |
| time/                   |             |
|    fps                  | 36          |
|    iterations           | 411         |
|    time_elapsed         | 45098       |
|    total_timesteps      | 1657152     |
| train/                  |             |
|    approx_kl            | 0.007648814 |
|    clip_fraction        | 0.074       |
|    clip_range           | 0.186       |
|    entropy_loss         | -0.453      |
|    explained_variance   | 0.0402      |
|    learning_rate        | 1.69e-05    |
|    loss                 | 0.0379      |
|    n_updates            | 4100        |
|    policy_gradient_loss | -0.00238    |
|    value_loss           | 122         |
-----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 5.91e+

# Evaluate

In [28]:
model = PPO.load('./train/best_model_62000.zip')

In [29]:
mean_reward, _ = evaluate_policy(model, env, render=True, n_eval_episodes=1)

d:\users\rushit\appdata\local\programs\python\python38\lib\site-packages\pyglet\image\codecs\wic.py:406: UserWarning: [WinError -2147417850] Cannot change thread mode after it is set
  warnings.warn(str(err))


In [38]:
mean_reward

4800.0

# Test The Model

In [30]:
obs = env.reset()
obs.shape

(1, 84, 84, 4)

In [31]:
env.step(model.predict(obs)[0])

(array([[[[  0,   0,  36,   0],
          [  0,   0,  36,   0],
          [  0,   0,  33,   0],
          ...,
          [  0,   0,  37,   0],
          [  0,   0,  34,   0],
          [  0,   0,  36,   0]],
 
         [[  0,   0,  36,   0],
          [  0,   0,  36,   0],
          [  0,   0,  35,   0],
          ...,
          [  0,   0,  36,   0],
          [  0,   0,  40,   0],
          [  0,   0,  40,   0]],
 
         [[  0,   0,  36,   0],
          [  0,   0,  36,   0],
          [  0,   0,  36,   0],
          ...,
          [  0,   0,  36,   0],
          [  0,   0,  36,   0],
          [  0,   0,  36,   0]],
 
         ...,
 
         [[  0,   0, 162,   0],
          [  0,   0, 159,   0],
          [  0,   0, 159,   0],
          ...,
          [  0,   0, 159,   0],
          [  0,   0, 159,   0],
          [  0,   0, 159,   0]],
 
         [[  0,   0, 162,   0],
          [  0,   0, 162,   0],
          [  0,   0, 162,   0],
          ...,
          [  0,   0, 162,   0],
 

In [32]:
# Reset game to starting state
obs = env.reset()
# Set flag to flase
done = False
for game in range(1): 
    while not done: 
        if done: 
            obs = env.reset()
        env.render()
        action = model.predict(obs)[0]
        obs, reward, done, info = env.step(action)
        time.sleep(0.01)
        if reward > 0 :
            print(reward)

[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]


[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]
[0.]


KeyboardInterrupt: 